# Imports

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns 

# Constants

In [3]:
token_spam_prob_file = "SpamData/03_Testing/prob-spam.txt"
token_ham_prob_file = "SpamData/03_Testing/prob-nonspam.txt"
token_all_prob_file = "SpamData/03_Testing/prob-all-tokens.txt"

test_feature_matrix = "SpamData/03_Testing/test-features.txt"
test_target_file = "SpamData/03_Testing/test-target.txt"

vocab_size = 2500